In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.5 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=fc43c55bc3cfcc13e3f0ff5db7b22939b49ecb739367adbf58270a4d3bf6678f
  Stored in directory: /Users/achethanreddy/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statistics import harmonic_mean
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('coursea_data.csv')
df.drop(['Unnamed: 0', 'course_organization'], axis=1, inplace=True)
df

,course_title,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,(ISC)² Systems Security Certified Practitioner...,SPECIALIZATION,4.7,Beginner,5.3k
1,A Crash Course in Causality: Inferring Causal...,COURSE,4.7,Intermediate,17k
2,A Crash Course in Data Science,COURSE,4.5,Mixed,130k
3,A Law Student's Toolkit,COURSE,4.7,Mixed,91k
4,A Life of Happiness and Fulfillment,COURSE,4.8,Mixed,320k
...,...,...,...,...,...
886,Программирование на Python,SPECIALIZATION,4.5,Intermediate,52k
887,Психолингвистика (Psycholinguistics),COURSE,4.8,Mixed,21k
888,Разработка интерфейсов: вёрстка и JavaScript,SPECIALIZATION,4.5,Intermediate,30k
889,Русский как иностранный,SPECIALIZATION,4.6,Intermediate,9.8k


In [4]:
df.course_students_enrolled.apply(lambda count : count[-1]).value_counts()

k    887
m      4
Name: course_students_enrolled, dtype: int64

In [5]:
df = df[df.course_students_enrolled.str.endswith('k')]

In [6]:
df['course_students_enrolled'] = df['course_students_enrolled'].apply(lambda enrolled : eval(enrolled[:-1]) * 1000)
df

/var/folders/c2/x_cgj8b10s38q85qqx_c88hr0000gn/T/ipykernel_70932/4222749396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course_students_enrolled'] = df['course_students_enrolled'].apply(lambda enrolled : eval(enrolled[:-1]) * 1000)


,course_title,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,(ISC)² Systems Security Certified Practitioner...,SPECIALIZATION,4.7,Beginner,5300.0
1,A Crash Course in Causality: Inferring Causal...,COURSE,4.7,Intermediate,17000.0
2,A Crash Course in Data Science,COURSE,4.5,Mixed,130000.0
3,A Law Student's Toolkit,COURSE,4.7,Mixed,91000.0
4,A Life of Happiness and Fulfillment,COURSE,4.8,Mixed,320000.0
...,...,...,...,...,...
886,Программирование на Python,SPECIALIZATION,4.5,Intermediate,52000.0
887,Психолингвистика (Psycholinguistics),COURSE,4.8,Mixed,21000.0
888,Разработка интерфейсов: вёрстка и JavaScript,SPECIALIZATION,4.5,Intermediate,30000.0
889,Русский как иностранный,SPECIALIZATION,4.6,Intermediate,9800.0


In [7]:
minmax_scaler = MinMaxScaler()
scaled_ratings = minmax_scaler.fit_transform(df[['course_rating','course_students_enrolled']])

In [8]:
df['course_rating'] = scaled_ratings[:,0]
df['course_students_enrolled'] = scaled_ratings[:,1]
df['overall_rating'] = df[['course_rating','course_students_enrolled']].apply(lambda row : harmonic_mean(row), axis=1)

/var/folders/c2/x_cgj8b10s38q85qqx_c88hr0000gn/T/ipykernel_70932/2019104726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course_rating'] = scaled_ratings[:,0]
/var/folders/c2/x_cgj8b10s38q85qqx_c88hr0000gn/T/ipykernel_70932/2019104726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course_students_enrolled'] = scaled_ratings[:,1]
/var/folders/c2/x_cgj8b10s38q85qqx_c88hr0000gn/T/ipykernel_70932/2019104726.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [9]:
df

,course_title,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,overall_rating
0,(ISC)² Systems Security Certified Practitioner...,SPECIALIZATION,0.823529,Beginner,0.004587,0.009122
1,A Crash Course in Causality: Inferring Causal...,COURSE,0.823529,Intermediate,0.018709,0.036586
2,A Crash Course in Data Science,COURSE,0.705882,Mixed,0.155100,0.254319
3,A Law Student's Toolkit,COURSE,0.823529,Mixed,0.108027,0.190999
4,A Life of Happiness and Fulfillment,COURSE,0.882353,Mixed,0.384430,0.535534
...,...,...,...,...,...,...
886,Программирование на Python,SPECIALIZATION,0.705882,Intermediate,0.060954,0.112217
887,Психолингвистика (Psycholinguistics),COURSE,0.882353,Mixed,0.023537,0.045850
888,Разработка интерфейсов: вёрстка и JavaScript,SPECIALIZATION,0.705882,Intermediate,0.034400,0.065602
889,Русский как иностранный,SPECIALIZATION,0.764706,Intermediate,0.010018,0.019777


In [10]:
df = df[df.course_title.apply(lambda title : detect(title) == 'en')]

In [11]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df.course_title)

In [12]:
def recommend_by_course_title (title, recomm_count=10) : 
    title_vector = vectorizer.transform([title])
    cosine_sim = cosine_similarity(vectors, title_vector)
    idx = np.argsort(np.array(cosine_sim[:,0]))[-recomm_count:]
    sdf = df.iloc[idx].sort_values(by='overall_rating', ascending=False)
    return sdf

In [17]:
dk = recommend_by_course_title('I want Machine Learing course')

In [18]:
dk

,course_title,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,overall_rating
563,Machine Learning,SPECIALIZATION,0.764706,Intermediate,0.348220,0.478533
28,Advanced Machine Learning,SPECIALIZATION,0.705882,Advanced,0.227520,0.344122
595,Mathematics for Machine Learning,SPECIALIZATION,0.764706,Beginner,0.179240,0.290410
2,A Crash Course in Data Science,COURSE,0.705882,Mixed,0.155100,0.254319
570,Machine Learning with Python,COURSE,0.823529,Intermediate,0.143030,0.243729
171,Crash Course on Python,COURSE,0.882353,Beginner,0.095957,0.173089
620,Nanotechnology: A Maker’s Course,COURSE,0.882353,Mixed,0.023537,0.045850
567,Machine Learning for All,COURSE,0.764706,Beginner,0.021123,0.041110
750,Stanford's Short Course on Breastfeeding,COURSE,0.823529,Beginner,0.018709,0.036586
569,Machine Learning for Trading,SPECIALIZATION,0.352941,Intermediate,0.016295,0.031151


In [19]:
dk.to_dict('records')

[{'course_title': 'Machine Learning',
  'course_Certificate_type': 'SPECIALIZATION',
  'course_rating': 0.7647058823529409,
  'course_difficulty': 'Intermediate',
  'course_students_enrolled': 0.348219674109837,
  'overall_rating': 0.4785326953748006},
 {'course_title': 'Advanced Machine Learning',
  'course_Certificate_type': 'SPECIALIZATION',
  'course_rating': 0.7058823529411764,
  'course_difficulty': 'Advanced',
  'course_students_enrolled': 0.22751961375980687,
  'overall_rating': 0.3441220096603659},
 {'course_title': 'Mathematics for Machine Learning',
  'course_Certificate_type': 'SPECIALIZATION',
  'course_rating': 0.7647058823529409,
  'course_difficulty': 'Beginner',
  'course_students_enrolled': 0.1792395896197948,
  'overall_rating': 0.2904099285445656},
 {'course_title': 'A Crash Course in Data Science',
  'course_Certificate_type': 'COURSE',
  'course_rating': 0.7058823529411764,
  'course_difficulty': 'Mixed',
  'course_students_enrolled': 0.1550995775497888,
  'overal